In [1]:
! pip install wheel
! pip install hazm
! pip install transformers
! pip install sentencepiece
! pip install numpy>=1.17

In [2]:
import random
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from tqdm import tqdm
import torch
import numpy as np
import time
import pandas as pd

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
!gdown 1MjK8syScLm4EpTL_4PUCXUjQqGuYHZtY

Downloading...
From: https://drive.google.com/uc?id=1MjK8syScLm4EpTL_4PUCXUjQqGuYHZtY
To: /content/test.jsonl
100% 399k/399k [00:00<00:00, 134MB/s]


In [5]:
import json

with open('test.jsonl', 'r') as json_file:
    json_list = list(json_file)

questions = {'math_and_logic':[],'common_knowledge':[],'literature':[]}
for json_str in json_list:
    result = json.loads(json_str)
    questions[result['category']].append({'question':result['question'],'candidates':result['candidates'],'answer':result['answer']})
    #print(f"result: {result}")


In [6]:
for key in questions:
  print(key,len(questions[key]))

math_and_logic 350
common_knowledge 350
literature 350


In [7]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

In [8]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-p

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def get_inference(prompt):
  messages = [
    {"role": "system", "content": ".تو یک دستیار ویرایشگر متن های فارسی هستی"},
    {"role": "user", "content": """{prompt}""".format(prompt = prompt)},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=500,
      eos_token_id=terminators,
      do_sample=False,
      temperature=0.5,
      top_p=0.9,
  )

  return(outputs[0]["generated_text"][len(prompt):])

In [11]:

PROMPT_FEW = """
در ادامه به تو یک سوال نشان داده می شود. تو باید از بین پاسخ های ارائه شده گزینه درست را انتخاب کنید.
به مثالهای زیر توجه کن:

  مثال 1:
      'question': 'عدد ۶ یک عدد کامل نامیده می\u200cشود، زیرا برابر است با مجموع مقسوم علیه\u200cهایش به استثنای خود عدد ۶ کدامیک از عددهای زیر عدد کامل است؟'
      'candidates': ['۱۲', '۲۸', '۳۶', '۱۶']
      'answer': '2'


  مثال 2:
      'question': ' از ابتدای تاریخ، کدام گزینه زیر انسانها را به همکاری با یکدیگر واداشته است؟'
      'candidates': ['ضرورتهای زندگی اجتماعی', 'اهداف سیاسی دولت- شهرها', 'تشکیل اجتماعات انسانی', 'فرهنگ پذیری']
      'answer': '1'


  مثال 3:
      'question': 'كدام كلمه جمع است'
      'candidates': ['پهلوان', 'مازندران', 'يزدان', 'ديوان']
      'answer': '4'


  مثال 4:
      'question': 'وسیع ترین کشور جهان کدام است؟',
      'candidates': ['آمریکا', 'کانادا', 'روسیه', 'چین'],
      'answer': '3'


  مثال 5:
      'question': 'در کدام گزینه، آثار موسوی گرما رودی تماما درست است؟',
      'candidates': ['سرود رگبار، دستچين، عبور، چمن لاله',
      'چمن لاله، خطّ خون، مثل درخت در شب باران، سرود رگبار',
      'در سايه سار نخل ولايت، از بودن و سرودن، خطّ خون، عبور',
       'تاناكجا، دستچين، در سايه سار نخل ولايت، از بودن و سرودن'],
      'answer': '1'



  فرمت خروجی:
  فرمت خروجی شما باید یک تاپل  باشد، که در آن هر تاپل از سوال و پاسخهای کاندید و پاسخ تشخیص داده شده تشکیل شده باشد.

  """

**math and logic**

In [18]:
questions['math_and_logic'][0]

{'question': 'تفاوت سن علیرضا و خواهرش A سال است B سال دیگر سن علیرضا دوبرابر سن امروز خواهرش خواهد بود .سن خواهر علیرضا کدام است؟',
 'candidates': ['2A', '2A+B', '3A+B', 'A-B'],
 'answer': '2'}

In [ ]:
import numpy as np

all_response = []

for index, row in enumerate(questions['math_and_logic']):
    print('i: ',index)
    quest = row['question']
    candid = row['candidates']
    ans = row['answer']

    prompt_arman = f"""
    شرح وظیفه:
    {PROMPT_FEW}

     سوال:
    {quest}

    گزینه ها:
    {ans}
    """
    response = get_inference(prompt_arman)

    all_response.append({'quest':quest ,'candid':candid, 'ans':ans , 'response':response})
    if index%10==0 :
        np.save(f'multi_choise_math_llama3_8b_arman_fewshot.npy', np.array(all_response, dtype=object))


i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
